Data imputation

In [66]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/data_clean.csv', sep=',', encoding='latin-1')
df.head()
# df.columns
# df.shape


,SEQN,RIDAGEYR,RIAGENDR,RIDRETH1,RIDRETH3,DMDCITZN,DMDEDUC2,MIALANG,DMDHHSIZ,INDHHIN2,...,LBDHDD,HIQ011,PAQ635,PAQ650,PAQ665,PAD680,PAQ706,PAQ710,LBXTC,LBXTR
0,73557,69,1,4,4,1.0,3.0,1.0,3,4.0,...,65.0,1,2,2,2,600.0,NaN,2.0,167.0,NaN
1,73558,54,1,3,3,1.0,3.0,1.0,4,7.0,...,50.0,2,2,2,2,540.0,NaN,4.0,170.0,NaN
2,73559,72,1,3,3,1.0,4.0,1.0,2,10.0,...,60.0,1,2,2,1,300.0,NaN,4.0,126.0,51.0
3,73561,73,2,3,3,1.0,5.0,1.0,2,15.0,...,85.0,1,2,2,2,480.0,NaN,1.0,201.0,75.0
4,73562,56,1,1,1,1.0,4.0,1.0,1,9.0,...,38.0,1,2,2,2,360.0,NaN,5.0,226.0,NaN


Data imputation for 3 columns

In [67]:
df = df[["BMXWT", "BMXHT", "BMXWAIST"]]

Convert dataframe to numpy array

In [74]:
# df[df.isnull()] = 0
df.shape
np_array = df.to_numpy(copy=True, na_value=0)

# for column in df.columns:
#     df_null_inx.append(df[df[column].isnull()].index.tolist())

# # df_null_inx

# for column in df.columns:
#     df[column].fillna((df[column].mean()), inplace=True)



Get non-missing indices

In [77]:
np_array
nonmissing_indices = np.flatnonzero(np_array > 0.1)
print(nonmissing_indices)
# u, s, v = np.linalg.svd(df)


[    0     1     2 ... 17304 17305 17306]


fill an array with available data

In [83]:
rating_matrix_ini = np.zeros(np_array.shape)
rating_matrix_ini.flat[nonmissing_indices] = np_array.flat[nonmissing_indices]


# rating_matrix_ini
# for column in np_array.shape[1]:
#     np_array[:,column] = 

Impute missing data

In [101]:
# rating_matrix_ini[6,:]
for column in range(np_array.shape[1]):
    np_array[np.where(np_array[:,column] == 0),column] = np.mean(np_array[:,column])



In [106]:
np_array.shape

(5769, 3)

Fit low rank model for rank 1

In [109]:
def fit_low_rank_model(rank,rating_matrix_ini,train_ind,train_data,n_iter,convergence_thresh,verbose):
    """Fit the low rank model. 
    Return the estimation of the low rank model - (n_movies * n_users) matrix

    Keyword arguments:
    rank -- the rank of low rank model
    rating_matrix_ini -- imputed initialization
    train_ind -- index of training data
    train_data -- ratings of training set
    n_iter -- the max number of iterations
    convergence_thresh -- the threshold of convergence to 0
    """
    previous_fitting_error = 100
    # Initialization
    low_rank_estimate = np.zeros(rating_matrix_ini.shape)
    # fill input data
    low_rank_estimate.flat[train_ind] = train_data
    # get the indexes of missing data
    missing_inds = np.where(low_rank_estimate.flat == 0)
    # fill missing data with imputed values
    low_rank_estimate.flat[missing_inds] = rating_matrix_ini.flat[missing_inds]

    
    for ind in range(n_iter):
        # Updates
        low_rank_estimate.flat[train_ind] = train_data
        u, s, v = np.linalg.svd(low_rank_estimate)
        s_matrix = s  * np.eye(len(s))
        low_rank_estimate = np.matmul(np.matmul(u[:,0:rank], s_matrix[0:rank,0:rank]), v[0:rank,:])
        # Compute error
        fitting_error = np.sqrt(((train_data - low_rank_estimate.flat[train_ind])**2).mean())
        if verbose:
            print("Iteration " + str(ind) + " Error: " + str(fitting_error))
        
        # Stopping criterion
        if (fitting_error <= convergence_thresh):
            print('converged, breaking')
            break
    return low_rank_estimate

n_iter = 5
convergence_thresh = 1e-4
verbose = True
rank = 1
train_data = np_array.flat[nonmissing_indices] 
estimate =fit_low_rank_model(rank,rating_matrix_ini,nonmissing_indices,train_data,n_iter,convergence_thresh,verbose)
print(estimate)

Iteration 0 Error: 13.827948862557633
Iteration 1 Error: 12.66187625529023
Iteration 2 Error: 12.540433132217627
Iteration 3 Error: 12.512156916173343
Iteration 4 Error: 12.500301721351864
[[ 83.71757929 168.15033455 100.75500642]
 [ 88.5602766  177.877099   106.58324468]
 [ 88.30001443 177.35435131 106.27001636]
 ...
 [ 83.52249682 167.75850307 100.52022256]
 [ 80.32002119 161.32619397  96.66600873]
 [ 83.1734518  167.05743124 100.10014313]]
